# Section: Encrypted Deep Learning


# Encrypted Computations in PySyft

In [1]:
!pip install syft
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

     |████████████████████████████████| 225kB 5.4MB/s 
     |████████████████████████████████| 256kB 45.0MB/s 
     |████████████████████████████████| 204kB 44.2MB/s 
     |████████████████████████████████| 81kB 27.1MB/s 
     |████████████████████████████████| 389kB 46.5MB/s 
     |████████████████████████████████| 450kB 45.7MB/s 
     |████████████████████████████████| 1.4MB 39.5MB/s 
     |████████████████████████████████| 276kB 44.3MB/s 
     |████████████████████████████████| 51kB 19.6MB/s 
     |████████████████████████████████| 122kB 42.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

W0713 16:36:07.274442 140615664809856 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0713 16:36:07.298275 140615664809856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
# we assigned one of the parties, secure_worker, to generate the random numbers 
# we have trust that the secure_worker, one of the parties, does not know Alice or Bob (they also do not know each other)
# use the parameter crypto_provider to assign on of the parties

x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker) # notice the number of the parties where the secrete is shared

In [0]:
bob._objects

{5832710301: tensor([1942124677794672934, 1386511262076046253, 1864651451166692784,
         2923358307548933517]),
 37155951963: tensor([1024298226974157009, 3757621307906408434, 3212090408686283635,
          585039451028585162])}

We have shared two secretes with three parties. Let's try some computations remotely on these secretes:

In [0]:
# adding secretes on remote machines
z = x + y
z.get() # returns and decodes

tensor([3, 1, 4, 4])

In [0]:
# subtracting secretes on remote machines

z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [0]:
# multiplying secretes on remote machines

z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [0]:
# boolean operations on remote machines
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [0]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [0]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [0]:
# double check that your implementation is using the fix_precision() function when dealing with float values
# reverse this function using the float_precision() function

x = th.tensor([1.2, 2.2, 3.2, 4.2])
y = th.tensor([2.1, -1.1, 1.1, 0.1])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get().float_precision()

tensor([3.3000, 1.1000, 4.3000, 4.3000])

In [0]:
z = x - y
z.get().float_precision()

tensor([-0.9000,  3.3000,  2.1000,  4.1000])

In [0]:
z = x * y
z.get().float_precision()

tensor([ 2.5200, -2.4200,  3.5200,  0.4200])

In [0]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [0]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [0]:
z = x == x
z.get().float_precision()

tensor([1., 1., 1., 1.])

# Lesson: Encrypted Deep Learning in PyTorch

### Build your algorithms and Model

In [0]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x



def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        


In [0]:
model = Net()

#train the model
train()

tensor(2.1571)
tensor(12.9535)
tensor(21.6353)
tensor(1.1473)
tensor(0.9781)
tensor(0.9570)
tensor(0.9398)
tensor(0.9207)
tensor(0.8990)
tensor(0.8741)
tensor(0.8455)
tensor(0.8128)
tensor(0.7757)
tensor(0.7359)
tensor(0.6929)
tensor(0.6448)
tensor(0.5935)
tensor(0.5378)
tensor(0.4791)
tensor(0.4152)


In [0]:
# run predictions
model(data)

tensor([[0.1881],
        [0.4219],
        [0.8541],
        [0.6796]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)
encrypted_model

Net(
  (fc1): Linear(in_features=2, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)

In [0]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:5747582235 -> bob:22955187796]
 	-> (Wrapper)>[PointerTensor | me:70178304243 -> alice:68052171563]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:43107561652 -> bob:71961852178]
 	-> (Wrapper)>[PointerTensor | me:86414278975 -> alice:68762703899]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:50688519605 -> bob:86048180237]
 	-> (Wrapper)>[PointerTensor | me:55780423456 -> alice:63836427461]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:23107485845 -> bob:14352544615]
 	-> 

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [0]:
encrypted_prediction.get().float_precision()

tensor([[0.1870],
        [0.4210],
        [0.8530],
        [0.6780]])

# Reuse the MNIST NN from the previous classes (firs week of classes) to train the classifier with a Secure Federated learning appraoch.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy
import helper

hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)

federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), 
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)



W0713 17:31:44.225317 139718200489856 hook.py:98] Torch was already hooked... skipping hooking process
W0713 17:31:44.227565 139718200489856 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0713 17:31:44.229123 139718200489856 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [0]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fconnected1 = nn.Linear(784, 128)
        self.fconnected2 = nn.Linear(128, 64)
        self.fconnected3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fconnected1(x))
        x = F.relu(self.fconnected2(x))
        x = F.log_softmax(self.fconnected3(x), dim=1)
        
        return x

In [0]:
def train(model, federated_train_loader, optimizer):
    for batch, (data, target) in enumerate(federated_train_loader): 
        model.send(data.location)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() 
        if batch % 30 == 0:
            loss = loss.get() 
            print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(batch * 64, len(federated_train_loader) * 64,
                100. * batch / len(federated_train_loader), loss.item()))

In [0]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:
model = Classifier()
optimizer = optim.SGD(model.parameters(), lr=0.3) 
train(model, federated_train_loader, optimizer)
test(model, test_loader)


[0/60032 (0%)]	Loss: 2.301429
[1920/60032 (3%)]	Loss: 0.642730
[3840/60032 (6%)]	Loss: 0.359475
[5760/60032 (10%)]	Loss: 0.374333
[7680/60032 (13%)]	Loss: 0.448258
[9600/60032 (16%)]	Loss: 0.309076
[11520/60032 (19%)]	Loss: 0.269373
[13440/60032 (22%)]	Loss: 0.350501
[15360/60032 (26%)]	Loss: 0.371032
[17280/60032 (29%)]	Loss: 0.522713
[19200/60032 (32%)]	Loss: 0.265876
[21120/60032 (35%)]	Loss: 0.312492
[23040/60032 (38%)]	Loss: 0.103462
[24960/60032 (42%)]	Loss: 0.298999
[26880/60032 (45%)]	Loss: 0.203673
[28800/60032 (48%)]	Loss: 0.182847
[30720/60032 (51%)]	Loss: 0.117810
[32640/60032 (54%)]	Loss: 0.120059
[34560/60032 (58%)]	Loss: 0.174427
[36480/60032 (61%)]	Loss: 0.076429
[38400/60032 (64%)]	Loss: 0.337325
[40320/60032 (67%)]	Loss: 0.093747
[42240/60032 (70%)]	Loss: 0.108687
[44160/60032 (74%)]	Loss: 0.088264
[46080/60032 (77%)]	Loss: 0.087266
[48000/60032 (80%)]	Loss: 0.102517
[49920/60032 (83%)]	Loss: 0.137768
[51840/60032 (86%)]	Loss: 0.026050
[53760/60032 (90%)]	Loss: 0.1956